In [39]:
import pandas as pd
import os
import csv

In [35]:
counties_2001 = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2012_counties.txt', sep='\t')
counties_2002 = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2002_counties.txt', sep='\t')

In [79]:
counties_data = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2001_counties.txt', sep='\t')

for file in os.listdir('county_data'):
    if file != 'Underlying Cause of Death, 1999-2017_2012_counties.txt':
        new_year = pd.read_csv('county_data/' + file, sep='\t', na_filter = False)
        counties_data = pd.concat([counties_data, new_year], axis=0)


/Users/Marta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [80]:
col_list = ['Notes', 'Population', 'State', 'State Code', 'Crude Rate']
for name in col_list:
    counties_data = counties_data.drop(columns = name)



In [83]:
new = counties_data["County"].str.split(",", n = 1, expand = True)

In [87]:
counties_data["County"]= new[0]
counties_data['State'] = new[1]

In [93]:
new_date = counties_data['Month Code'].str.split('/', n=1, expand=True)

In [94]:
counties_data['Year']=new_date[0]
counties_data['Month']=new_date[1]

In [98]:
state_data = pd.read_csv('Underlying Cause of Death, 1999-2017_allyears_states.txt', sep='\t')

In [101]:
state_pops_2000 = pd.read_csv('co-est00int-agesex-5yr.csv', encoding='latin-1')

In [103]:
state_pops_2010 = pd.read_csv('co-est2018-alldata.csv', encoding='latin-1')

In [108]:
new_county_name = state_pops_2000['CTYNAME'].str.split(' ', n=1, expand=True)
state_pops_2000['County'] = new_county_name[0]

In [111]:
new_county_name = state_pops_2010['CTYNAME'].str.split(' ', n=1, expand=True)
state_pops_2010['County'] = new_county_name[0]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2018,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,County
0,40,3,6,1,0,Alabama,Alabama,4779736,4780138,4785448,...,1.171362,0.516888,1.234191,1.607294,0.619874,0.646064,0.817666,1.079070,1.856398,Alabama
1,50,3,6,1,1,Alabama,Autauga County,54571,54574,54754,...,0.666403,6.002073,-6.119262,-3.885124,1.971001,-1.859380,5.250727,1.029950,0.630381,Autauga
2,50,3,6,1,3,Alabama,Baldwin County,182265,182264,183111,...,24.298662,16.345147,17.638173,22.876199,20.239802,17.932091,21.484903,22.534622,24.846682,Baldwin
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27330,...,-9.072923,0.256035,-6.786002,-8.093425,-5.102231,-15.531931,-18.152860,-25.030896,-8.593297,Barbour
4,50,3,6,1,7,Alabama,Bibb County,22915,22920,22872,...,-7.029251,-5.041759,-3.787629,-5.666224,1.065317,1.507660,-0.753246,-3.146396,-6.584362,Bibb
5,50,3,6,1,9,Alabama,Blount County,57322,57321,57373,...,-0.172910,0.243633,-1.633022,-0.295188,-1.980336,-1.286475,-1.025739,6.363573,-0.034582,Blount
6,50,3,6,1,11,Alabama,Bullock County,10914,10911,10878,...,-2.953628,-22.546973,-7.047547,-6.427829,9.521116,-24.590553,-3.079736,-21.014739,-3.150537,Bullock
7,50,3,6,1,13,Alabama,Butler County,20947,20943,20942,...,-10.965688,-3.682449,-7.795020,-14.417574,0.442076,-6.268664,-6.118642,-5.209507,-10.106625,Butler
8,50,3,6,1,15,Alabama,Calhoun County,118572,118594,118477,...,-2.210176,-6.306238,-4.535128,-5.218180,-3.999381,-3.308412,-3.712796,-1.837323,-1.589929,Calhoun
9,50,3,6,1,17,Alabama,Chambers County,34215,34171,34122,...,-1.721758,-1.437962,4.549858,2.549563,-3.787267,1.588539,-5.399982,2.547054,-1.128048,Chambers
